Выполняли: Максимкина Анастасия, Корнева Дарья

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Сначала читаем данные и создаем DataFrame:

In [ ]:
import pandas as pd
ice = pd.read_csv('../input/seaice/seaice.txt', delim_whitespace = True)
print ('shape:', ice.shape)
ice.head()

In [ ]:

ice2 = ice[ice.data_type != '-9999']

In [ ]:
import seaborn as sns
sns.lmplot("mo", "extent", ice2)#иллюстрируем связь между mo и extent 

Нормализируем данные и визуализируем линейные отношения:

In [ ]:
month_means=ice2[["extent","mo"]].groupby("mo").mean()
for i in range(12):
    ice2.extent[ice2.mo==i+1]=100*(ice2.extent[ice2.mo==i+1]-month_means.extent[i+1])/month_means.extent.mean()
sns.lmplot("mo", "extent", ice2)

In [ ]:
sns.lmplot("year", "extent", ice2)

Устанавливаем состояние оценщика на основе обучающих данных, используя функцию подбора (fit)

In [ ]:
from sklearn. linear_model import LinearRegression
est = LinearRegression(fit_intercept = True)


Подгоняем регресионную модель, используя year в качестве переменных (x) и значения extent в качестве ответа (y)

In [ ]:
x = ice2[[ 'year']]
y = ice2[[ 'extent']]
est.fit(x, y)
print ("Coefficients:", est.coef_)
print("Intercept:", est.intercept_)

In [ ]:
from sklearn import metrics
y_hat = est.predict(x)
print ("MSE:", metrics.mean_squared_error(y_hat , y))
print ("R^2:", metrics.r2_score(y_hat , y))
print ('var:', y.var())

Учитывая модель, также можно предсказать значение extent для определенного года.

In [ ]:
x = [[2025]]
y_hat = est.predict(x)
m = 1 #January
y_hat = (y_hat*ice2.mo.mean()/100) + ice2.mo[m]
print ("Prediction of extent for January 2025 (in millions of square km):", y_hat)

проанализируем связь между ценой и площадью комнаты следующим образом:

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
rooms = pd.read_excel('../input/room12/ROOM.XLS')
sns.lmplot("price", "livsq", rooms)


Функция lmplot позволяет изменить порядок модели

In [ ]:
sns.lmplot("price", "livsq", rooms, order = 2)


Рассматриваем только 6 переменных

In [ ]:
df = pd.DataFrame(rooms[['distc','distm','totsq','kitsq',
'livsq','price']])
corrmat = df.corr()
sns.heatmap(corrmat,vmax = .8, square = True)


График корреляции: тепловая карта, представляющая корреляцию между шестью парами переменных в наборе данных рекламных объявлений г. Москва

Построим диаграмму рассеивания

In [ ]:
df = rooms[['distc', 'totsq', 'livsq', 'price']]
print(df.shape)
pd.plotting.scatter_matrix(df , figsize = (10.0, 10.0))

Разбиваем данные на обучающий набор и тестовый набор и вычисляем показатель линейной регрессии

In [ ]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
rooms.drop(['region', 'date'], axis='columns', inplace=True)
train_size = int(rooms.shape[0]/2)
price = rooms.price
rooms.drop(['price'], axis='columns', inplace=True)
X_train = rooms[:train_size]
X_test = rooms[train_size:]
y_train = price[:train_size]
y_test = price[train_size:]
print ('Training and testing set sizes',rooms.shape, rooms.shape)
regr = LinearRegression()
regr.fit(X_train , y_train)
print ('Coeff and intercept:',regr.coef_ , regr. intercept_)
print ('Testing Score:', regr.score (X_test , y_test))
print ('Training MSE: ',np.mean((regr. predict(X_train) -
y_train )**2))
print ('Testing MSE: ',np.mean((regr .predict(X_test ) -
y_test)**2))


Видно, что все полученные коэффициенты отличаются от нуля, что означает, что ни одна переменная не отбрасывается.

Для построения разреженной модели используем регрессор LASSO, прогнозирование цены

In [ ]:
regr_lasso = linear_model.Lasso(alpha = .3)
regr_lasso.fit(X_train , y_train)
print ('Coeff and intercept:',regr_lasso.coef_)
print ('Tesing Score: ', regr_lasso.score(X_test ,y_test))
print ('Training MSE:',
np.mean((regr_lasso.predict(X_train) - y_train)**2))
print ('Testing MSE:',
np.mean((regr_lasso.predict(X_test) - y_test)**2))

Теперь можно видеть, что результат подбора модели для набора разреженных коэффициентов лишь немного лучше, чем прежде

С помощью результата LASSO также можно выделить наиболее важные факторы для определения цены квартиры в Москве на основе значений коэффициентов:


In [ ]:
ind = np.argsort(np.abs(regr_lasso.coef_))
names = np.array(list(rooms))
print ('Ordered variable (from less to more important):',
names[ind])

визуализируем целевой и прогнозируемый отклики в точечной диаграмме, чтобы оценить прогноз

In [ ]:
import matplotlib.pyplot as plt
clf = LinearRegression()
clf.fit(rooms, price)
predicted = clf.predict(rooms)
plt.scatter(price, predicted , alpha = 0.5)
plt.plot([0, 70], [0, 70], '--k')
plt.axis('tight')
plt.xlabel('True price')
plt.ylabel('Predicted price')


показываем с какой вероятностью количество выкуренных сигарет за день влияет на развитие заболеваний сердца

In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
data = pd.read_csv('../input/framingham-heart-study-dataset/framingham.csv')
s = data[['cigsPerDay','TenYearCHD']]
s=s.dropna()
x = s.cigsPerDay
x = x[:, np.newaxis]
y = s.TenYearCHD#10-летний риск ишемической болезни сердца ИБС 
#(двоичный код: «1» означает «Да», «0» означает«Нет»)
logreg = LogisticRegression()
logreg.fit(x, y)
X_test = np.linspace(-200, 700, 300)
def lr_model(x):
 return 1 / (1+np.exp(-x))
loss = lr_model(X_test*logreg.coef_ + logreg.intercept_).ravel()
X_test2 = X_test[:,np.newaxis]
losspred = logreg.predict(X_test2)


In [ ]:
plt.scatter(x.ravel(), y,
color = 'black',
s = 100, zorder = 20,
alpha = 0.03)
plt.plot(X_test , loss , color = 'blue', linewidth = 3)
plt.plot(X_test , losspred , color = 'red', linewidth = 3)

Линейная регрессия (синяя) и логистическая регрессия
(красная) для подбора набора данных (черные точки), обычно
распределенных по значениям 0 и 1 y.
